## Solvation energy of a Zika virus

We present a more challenging problem that studies the solvation energy of the Zika virus (PDB code 6CO8).
We ran this experiment on a single CPU node of Pegasus, and the raw result files are located at `/repro-pack/runs/6CO8`.
This notebook shows how we generated the results in section 3.5, excluding the surface plot produced by ParaView.
We downloaded the molecular structure from the Protein Data Bank (PDB), parameterized it with the amber force field, and generated a mesh on the solvent-excluded surface using Nanoshaper.
The prepared structure contains about 1.6 million atoms and our mesh has around 10 million boundary elements.
In this experiment, 3 quadrature points were used for regular Galerkin integrals over disjoint elements. The FMM expansion order was 4 and the tolerance of GMRES was $10^{-4}$.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from bempp_pbs.postprocess import PLOT_PARAMS, get_df

**load results**

Similar to the previous study, we obtained three sets of results:
- `direct`: direct formulation with a block-diagonal preconditioner
- `derivative`: derivative formulation with a mass-lumping preconditioner
- `derivative_mass_matrix`: derivative formulation with a mass-matrix preconditioner

In [2]:
direct_df = get_df('../runs/6CO8/direct/', formulation='direct', skip4=True)
derivative_df = get_df('../runs/6CO8/derivative_ex/', formulation='derivative', skip4=True)
derivative_mass_matrix_df = get_df('../runs/6CO8/derivative_ex_mass_matrix/', formulation='derivative', skip4=True)

zika_df = pd.concat([direct_df, derivative_df, derivative_mass_matrix_df])
zika_df.index = ['direct', 'derivative', 'derivative_mass_matrix']
zika_df

,t_total_assembly,t_total_gmres,num_iter,e_solv [kcal/Mol],memory [GB],t_fmm_init,t_singular_assembler,t_assemble_sparse,t_assembly_other,t_singular_correction,t_laplace,t_helmholtz,t_avg_laplace,t_avg_helmholtz,t_gmres_other
direct,1534.450776,9470.869895,105,-116587.549968,109.725492,156.044683,466.436640,25.74,886.229454,791.160887,2130.213909,5670.189209,4.977135,13.248106,879.305890
derivative,3553.932997,4816.362144,18,-116254.861495,151.950672,148.309193,651.362527,98.26,2656.001277,367.972429,791.603687,2897.297981,4.947523,13.169536,759.488048
derivative_mass_matrix,7766.815987,5009.352236,17,-116255.417177,174.958108,151.262922,651.984174,98.07,6865.498891,460.615885,776.339616,2796.233184,5.107497,13.379106,976.163552


To verify our result, we ran the same case with PyGBe using the direct formulation on a workstation with a 24-core CPU. The solvation energy computed from PyGBe is -117261.125498 [kcal/mol].

-------

### Appendix:

To justify our choice of regular quadrature order, we ran the same case using the direct formulation with various quadrature orders. The results are summarized in the table below. These runs were performed on a different hardware, hence these timings are not comparable to the timings reported in the table above.

| quadrature order |  # quadrature points | expansion order | tol  | e_solv     | gmres time (s) | # iter |
|:----------------:|----------------------|-----------------|------|------------|----------------|--------|
|         1        |          1           |       4         | 1e-4 | -123693.42 | 4149.30        | 105    |
|         2        |          3           |       4         | 1e-4 | -116587.55 | 10786.23       | 105    |
|         3        |          4           |       4         | 1e-4 | -116491.99 | 13273.08       | 105    |
|         4        |          6           |       4         | 1e-4 | -116746.19 | 20891.13       | 105    |

With 3 quadrature points (a quadrature order of 2), the solvation energy is -116587.55 [kcal/mol]. With 6 quadrature points (a quadrature order of 4), the solvation energy is -116746.19 [kcal/mol]. The relative difference between the two is 0.14%, indicating that using 3 quadrature points is adequate for the solvation energy calculation.